In [5]:
import os
import json
import base64
from uuid import uuid4
import random
from random import randint

# 3rd party
import openai
from openai import OpenAI
from dotenv import find_dotenv, load_dotenv
from rich.console import Console
import psycopg2

In [19]:
console = Console()
load_dotenv(find_dotenv())


def get_test_key():

    if load_dotenv():
        OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
        console.print(
            f".env file found with OPENAI_API_KEY: [dark_orange]{OPENAI_API_KEY[:12]}[/dark_orange]"
        )
    else:
        console.print(
            "[red]Caution: No environment variables found. Please create .env file in the root directory or add environment variables in the .env file[/red]"
        )

    return OPENAI_API_KEY

In [ ]:
OPENAI_API_KEY = get_test_key()
openai.api_key = OPENAI_API_KEY
client = OpenAI()

In [ ]:
PG_URL = ""


def get_db():
    return psycopg2.connect(PG_URL)


conn = get_db()
if conn:
    print(f"Conn: {conn}\n")
else:
    print("NO CONNECTION\n")

In [22]:
cursor = conn.cursor()

In [ ]:
sql_pgvector = """CREATE EXTENSION IF NOT EXISTS vector;"""
try:
    cursor.execute(sql_pgvector)
    conn.commit()
    print("PGVECTOR extension created successfully........\n")

except Exception as e:
    print(f"Error {e}")

In [24]:
TABLE_NAME = "publisher2"
# TABLE_NAME = "tbl_doc_elements"

In [7]:
sql01 = """CREATE TABLE IF NOT EXISTS tbl_doc_elements (
    id uuid NOT NULL DEFAULT gen_random_uuid(),
    file_id uuid NOT NULL,
    element_id VARCHAR(150),
    element_type VARCHAR(50),
    element_text TEXT,
    parent_id VARCHAR(150),
    page_number INTEGER,
    filename VARCHAR(150),
    filetype VARCHAR(50),
    processed_text TEXT,
    keywords TEXT [],
    ner TEXT [], 
    embedding VECTOR(1536),
    image_base64 TEXT,
    created_at TIMESTAMPTZ NOT NULL DEFAULT NOW(),
    CONSTRAINT doc_elements_pk PRIMARY KEY (id)
);"""

In [ ]:
try:
    cursor.execute(sql01)
    conn.commit()
    print(f"{TABLE_NAME} table created successfully........\n")

except Exception as e:
    print(f"Error {e}")

In [ ]:
OPENAI_API_KEY = get_test_key()
# Set up your OpenAI API key

# Choose a model

In [ ]:
sql = f"""CREATE TABLE IF NOT EXISTS {TABLE_NAME} (
				publisher_id SERIAL PRIMARY KEY,
				publisher_name VARCHAR(255) NOT NULL,
				publisher_estd INT,
				publsiher_location VARCHAR(255),
				publsiher_type VARCHAR(255)

)"""

try:
    cursor.execute(sql)
    conn.commit()
    print(f"{TABLE_NAME} table created successfully........\n")

except Exception as e:
    print(f"Error {e}")

In [ ]:
try:
    postgres_insert_query = f""" INSERT INTO {TABLE_NAME} (publisher_id,
	publisher_name, publisher_estd, publsiher_location, publsiher_type)
	VALUES (%s,%s,%s,%s,%s)"""

    record_to_insert = [
        (random.randint(1000, 9999), "Packt", 1950, "chennai", "books"),
        (random.randint(1000, 9999), "Springer", 1950, "chennai", "books"),
        (random.randint(1000, 9999), "Springer", 1950, "chennai", "articles"),
        (random.randint(1000, 9999), "Oxford", 1950, "chennai", "all"),
        (random.randint(1000, 9999), "MIT", 1950, "chennai", "books"),
    ]
    for i in record_to_insert:
        cursor.execute(postgres_insert_query, i)

        conn.commit()
        count = cursor.rowcount
        print(
            count,
            f"Record inserted successfully into {TABLE_NAME} table",
        )

except (Exception, psycopg2.Error) as error:
    print("Failed to insert record into publisher table", error)

In [ ]:
try:
    postgreSQL_select_Query = f"select * from {TABLE_NAME}"

    cursor.execute(postgreSQL_select_Query)
    print(f"\nSelecting rows from {TABLE_NAME} table using cursor.fetchall")
    records = cursor.fetchall()

    for row in records:
        # print("publisher_id = ",row[0])
        # print("publisher_name = ", row[1])
        # print("publisher_estd = ", row[2])
        # print("publisher_location = ", row[3])
        # print("publisher_type = ", row[4], "\n")
        print(row)

except (Exception, psycopg2.Error) as error:
    print("Error while fetching data from PostgreSQL", error)

In [ ]:
cursor.close()
conn.close()
print("PostgreSQL connection is closed")